**Extract Topics from files**

In [1]:
import numpy as np
import pandas as pd
import gensim
import spacy
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import os
import sys
import glob

import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\svenkata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\svenkata\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

import matplotlib.pyplot as plt


In [2]:
websites_txt = 'C:/Users/svenkata/Documents/websites_txt/'
a_txt = 'C:/Users/svenkata/Documents/a_txt/'

*Conventional algorithms are often biased towards the majority class, not taking the data distribution into consideration*

*Many times, the minority classes are treated as outliers*

In [188]:
stop = stopwords.words('english')

exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [189]:
def clean(doc):
    stop_free = ' '.join([i for i in doc if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalised = ''.join(lemma.lemmatize(word) for word in punc_free)
    
    return normalised

In [155]:
'''
doc = ['easy',' ways', 'say', 'i\'d',' love',' but', 'have', 'floss']
clean(doc)
'''

['easy', 'ways', 'say', 'id', 'love', 'but', 'floss']

In [144]:
##I know this is very unprofessional but, string manipulation is very difficult otherwise!!
def replace_chars(path):
    to_replace = ['\\\\','\\']
    string=''
    for ch in path:
        if ch in to_replace:
            ch = '/'
        elif ch == '\v':
            ch = '/v'
        elif ch == '\b':
            ch = '/b'
        elif ch == '\t':
            ch = '/t'
        elif ch == '\a':
            ch = '/a'
        elif ch == '\f':
            ch = '/f'
        elif ch == '\n':
            ch = '/n'
        elif ch == '\r':
            ch = '\r'
        elif ch == '\h':
            ch = '/h'
        elif ch == '\o':
            ch = '/o'
        string = string+ch
    return string

In [190]:
import funcy as fp
## Something quick and dirty......
EMAIL_REGEX = re.compile(r'[a-z0-9\.\+_-]+@[a-z0-9\.\+_-]+\.[a-z]*')
FILTER_REGEX = re.compile(r"[^a-z '#]")
TOKEN_MAPPINGS = [(EMAIL_REGEX,"#email"),(FILTER_REGEX,' ')]

def tokenize_line(line):
    res = line.lower()
    for regexp, replacement in TOKEN_MAPPINGS:
        res = regexp.sub(replacement,res)
    return res.split()

def tokenize(lines, token_size_filter = 2):
    tokens = fp.mapcat(tokenize_line,lines)
    return [t for t in tokens if len(t) > token_size_filter]
   

def load_doc(folder_path):
    doc_list =[]
    for subdir, dirnames, files  in os.walk(folder_path):         
        for filename in files:             
            if (filename.split('.')[-1] =='txt'):
                #print(filename)
                filepath = subdir+os.sep+filename
                documentname = replace_chars(filepath)
                try:
                    with open(filepath, encoding='UTF-8', errors='ignore') as f:
                        doc = f.readlines()                          
                        tokens = tokenize(doc)
                        tokens = clean(tokens)
                        
                        
                    doc_list.append({'documentname': documentname, 'tokens':tokens  })
                except :
                    print(filepath)
            else:
                continue     
             
    doc_df = pd.DataFrame(doc_list)
    
    return doc_df



In [191]:
docs_websites = load_doc(websites_txt)

C:/Users/svenkata/Documents/websites_txt/Webteam\blogs\images from bloggers\Martin Willis blog pics\In reviewing the correspondence it is clear that the substance of your complaint regarding the wall boards is a continuation of the issues raised in your previous complaints.txt
C:/Users/svenkata/Documents/websites_txt/Webteam\Editorial\FOI request files 2016-12-12\Payments to he Advisory Panel on Public Sector Information [APPSI] and the Advisory Council on National Records and Archives [ACNRA] to the Confederation of British Industry and its subsidiaries.txt


In [192]:
docs_websites.head()

documentname  \
0  C:/Users/svenkata/Documents/websites_txt/Angel...   
1  C:/Users/svenkata/Documents/websites_txt/Angel...   
2  C:/Users/svenkata/Documents/websites_txt/Angel...   
3  C:/Users/svenkata/Documents/websites_txt/Angel...   
4  C:/Users/svenkata/Documents/websites_txt/Angel...   

                                              tokens  
0  sheet archives sector pages broken links octob...  
1  sheet cake contributors yes kind cake vegan gl...  
2  https media nationalarchives gov index php yea...  
3  blog front end https blog nationalarchives gov...  
4  https media nationalarchives gov index php col...

In [193]:
# Consists of contents of websites files as tokens for developing NLP models
docs_websites.to_pickle('H:/AI_for_Selection/websites_contents_NLP.pkl') # 

In [194]:
docs_a = load_doc(a_txt)
docs_a.to_pickle('H:/AI_for_Selection/a_contents_NLP.pkl')

C:/Users/svenkata/Documents/a_txt/20032008fp\RMDaILUa\C_3\DfID\A_3\PRAaD\AISU note of decisions from meeting with Middle East Centre and British Empire and Commonwealth Museum forwarded to DFID for confirmation that proposed solution is acceptable[A121439.1].txt
C:/Users/svenkata/Documents/a_txt/ASD\Cio\RSE\PHC0042\PHCIoSLO2015\Portsmouth Library and Archive Service - Inspection of Southsea Outstore 24th February 2015  - Advisory Letter   25th January 2016 - Postponing appointment pending accreditation[A4185929.1].txt
C:/Users/svenkata/Documents/a_txt/ASD\CS\AtA\AtApm\CaIPODPaICO\DIGITAL MEETS CULTURE - source for digital copyright contacts_activities_ RE_ published Fwd_ _AN INVITATION_ UKAD Forum - Thursday 27 March 2014, The National Archives [UNCLASSIFIED][A3529985.1].txt
C:/Users/svenkata/Documents/a_txt/ASD\Rwgan\Gb\NLHFHLF\CC_7\Collecting Cultures - response to request for second meeting - separate strand for discussing archive purchasing behaviour_ RE_ Next Collecting Cultures w

# Model Building

**Read Preprocessed Data into Data frame**

In [3]:
docs_a_df = pd.read_pickle('H:/AI_for_Selection/a_contents_NLP.pkl')# This file consists of the contents of the files (structured data from Objective)

In [4]:
a_meta_df = pd.read_pickle('H:/AI_for_Selection/metadata_with_filesize_lastmodified.pkl') # This file consists of processed metadata of structured data

txt_extensions = ['pdf','rtf','txt','msg','doc','docx','xls', 'xlsx','mbx']

a_meta_df=a_meta_df[a_meta_df.fileextension.isin(txt_extensions)]
a_meta_df = a_meta_df[['fileextension', 'disposal_schedule', 'repository','file_path' ,'ret_schedule','file_size']].copy()

In [5]:
websites_meta_df = pd.read_pickle('H:/AI_for_Selection/Websites_processed.pkl')# This file consists of metadata of processed websites data

In [6]:
docs_websites = pd.read_pickle('H:/AI_for_Selection/websites_contents_NLP.pkl') # This file consists of the contents of the files (unstructred X-drive data)
                    

In [41]:
docs_websites.head(1)

documentname  \
0  C:/Users/svenkata/Documents/websites_txt/Angela-Hill/archives-sector-dec-17.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

**Merge contents file with metadata file on the target column (retention schedule)**

In [7]:
pd.set_option('display.max_colwidth',None) # to view the whole width of the cells in a dataframe

In [57]:
print(docs_a_df[:1]['documentname'])

0    C:/Users/svenkata/Documents/a_txt/20032008fp/Am/Cs/Es/L_5/LNAS/MDR 1 How to use the paper MDR[A905701.3].txt
Name: documentname, dtype: object


In [58]:
##function to extract file's absolute path and filename with no extension
def extract_absolutePath_removeExtension(topfolderpath, filename):
    filename = filename.replace(topfolderpath,'')
    words = filename.split('.')
    return '.'.join(words[:-1])

In [59]:
docs_a_df['abs_filepath'] = docs_a_df['documentname'].apply(lambda x: extract_absolutePath_removeExtension('C:/Users/svenkata/Documents/a_txt/',x))

In [60]:
print(docs_a_df[:1])

                                                                                                   documentname  \
0  C:/Users/svenkata/Documents/a_txt/20032008fp/Am/Cs/Es/L_5/LNAS/MDR 1 How to use the paper MDR[A905701.3].txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [61]:
# check how the file path is represented in structured file
print(a_meta_df[:1])

  fileextension                                           disposal_schedule  \
0           xls  24 Projects - Full Projects (Close file when Project ends)   

           repository  \
0  Strategic Projects   

                                                                                          file_path  \
0  PPDaCM/PD/PC_1/20YRaCS/GE_1/2012RTR/RTRS2012DS/The National Archives RTR 09-2012[A3109716.2].xls   

  ret_schedule  file_size  \
0           24     225280   

                                                                                   abs_filepath  
0  PPDaCM/PD/PC_1/20YRaCS/GE_1/2012RTR/RTRS2012DS/The National Archives RTR 09-2012[A3109716.2]  


In [62]:
a_meta_df['abs_filepath']  = a_meta_df['file_path'].apply(lambda x: extract_absolutePath_removeExtension('', x))

In [63]:
print(a_meta_df[:1])

  fileextension                                           disposal_schedule  \
0           xls  24 Projects - Full Projects (Close file when Project ends)   

           repository  \
0  Strategic Projects   

                                                                                          file_path  \
0  PPDaCM/PD/PC_1/20YRaCS/GE_1/2012RTR/RTRS2012DS/The National Archives RTR 09-2012[A3109716.2].xls   

  ret_schedule  file_size  \
0           24     225280   

                                                                                   abs_filepath  
0  PPDaCM/PD/PC_1/20YRaCS/GE_1/2012RTR/RTRS2012DS/The National Archives RTR 09-2012[A3109716.2]  


**Merge doc contents with metadata on abs_filepath**

In [64]:
X = pd.merge(docs_a_df, a_meta_df, on='abs_filepath')

In [65]:
print(X.columns)

Index(['documentname', 'tokens', 'abs_filepath', 'fileextension',
       'disposal_schedule', 'repository', 'file_path', 'ret_schedule',
       'file_size'],
      dtype='object')


**add a numerical category_id to avoid string type ret_schedule**

In [67]:
X['category_id'] = X['ret_schedule'].factorize()[0]
category_id_df = X[['disposal_schedule','category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id','disposal_schedule']].values)
X[:1]

documentname  \
0  C:/Users/svenkata/Documents/a_txt/20032008fp/Am/Cs/Es/L_5/LNAS/MDR 1 How to use the paper MDR[A905701.3].txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   tokens  \
0  mdr use manorial documents register mdr manorial documents register mdr mdr index manorial records provides brief descriptions documents details locations mdr mainly paper index details counties wales three ridings yorkshire hampshire isle wight norfolk surrey middlesex revised updated computerised searched online www mdr nationalarchives gov mdr see nra non computerised counties use paper indexes mdr paper indexes mdr held research enquiries room divided two parts parish index manor index parish index parish index identifies names manors associated parishes two always identical arranged county alphabetically parish name within county information contained slip parish name middle slip manor name located top right hand corner manors may known variety names records listed one name manor index slip two manor names top right hand corner name preceded see standard manor name may one slip associated parish giving several manor names research possible parish index may include reference manor records indexed manor index indicates manor existed records survived location remains unknown please aware parish include one manors manors cross parish boundaries parts manors exist separately main body manor different parishes counties manor name known manor index consulted manor index manor index arrange

In [68]:
print('Length of X:',len(X))
# just checking
print(X.category_id.value_counts())
print(X.ret_schedule.value_counts())

Length of X: 93824
7     18741
11    13110
0     10786
6      8973
12     8047
1      7100
10     5115
4      2762
14     2713
3      2686
13     2679
9      2573
5      2298
8      1909
2      1612
18     1486
15      925
16      250
17       58
19        1
Name: category_id, dtype: int64
24     18741
02     13110
33     10786
05      8973
04      8047
23      7100
03      5115
20      2762
11      2713
21      2686
07      2679
27      2573
32      2298
28      1909
16      1612
24b     1486
10       925
25       250
06        58
24a        1
Name: ret_schedule, dtype: int64


In [69]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,6))
X.groupby('ret_schedule').tokens.count().plot.bar(ylim=0)
plt.show()

# Working with Text docs

**Naive Bayes**

In [72]:
X1 = X.tokens
y = X.ret_schedule
X_train, X_test, y_train, y_test = train_test_split(X1,y,test_size = 0.3, random_state=42)

In [73]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer


In [74]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [78]:
%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred,y_test))
print(classification_report(y_test, y_pred))

Wall time: 0 ns
accuracy 0.5938965468239307
              precision    recall  f1-score   support

          02       0.44      0.89      0.59      4011
          03       0.93      0.36      0.52      1519
          04       0.96      0.77      0.85      2415
          05       0.95      0.48      0.64      2685
          06       0.00      0.00      0.00        21
          07       1.00      0.01      0.02       786
          10       0.00      0.00      0.00       294
          11       1.00      0.10      0.18       792
          16       1.00      0.00      0.01       487
          20       0.99      0.15      0.26       813
          21       1.00      0.00      0.01       775
          23       0.99      0.42      0.59      2085
          24       0.50      0.98      0.66      5787
         24b       1.00      0.02      0.04       449
          25       0.00      0.00      0.00        70
          27       0.93      0.11      0.20       747
          28       0.97      0.14    

**Linear Support Vector Machine**

In [80]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', SGDClassifier(loss= 'hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
              ])
sgd.fit(X_train, y_train)

%time

y_pred = sgd.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred,y_test))
print(classification_report(y_test, y_pred))

Wall time: 0 ns
accuracy 0.8010515844820236
              precision    recall  f1-score   support

          02       0.80      0.75      0.77      4011
          03       0.73      0.94      0.82      1519
          04       0.92      0.88      0.90      2415
          05       0.83      0.80      0.81      2685
          06       1.00      0.10      0.17        21
          07       0.84      0.60      0.70       786
          10       0.87      0.30      0.45       294
          11       0.52      0.63      0.57       792
          16       0.92      0.89      0.90       487
          20       0.82      0.70      0.76       813
          21       0.86      0.48      0.62       775
          23       0.86      0.87      0.87      2085
          24       0.81      0.91      0.86      5787
         24a       0.00      0.00      0.00         0
         24b       0.88      0.95      0.91       449
          25       0.89      0.24      0.38        70
          27       0.82      0.65    

**Logistic Regression**

In [83]:
from sklearn.linear_model import LogisticRegression

lr = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
lr.fit(X_train, y_train)

%time

y_pred = lr.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

Wall time: 15.6 ms
accuracy 0.8967955094500497
              precision    recall  f1-score   support

          02       0.87      0.87      0.87      4011
          03       0.83      0.87      0.85      1519
          04       0.97      0.95      0.96      2415
          05       0.92      0.91      0.91      2685
          06       0.85      0.52      0.65        21
          07       0.85      0.83      0.84       786
          10       0.87      0.76      0.81       294
          11       0.60      0.89      0.71       792
          16       1.00      0.95      0.97       487
          20       0.88      0.85      0.87       813
          21       0.86      0.84      0.85       775
          23       0.93      0.94      0.93      2085
          24       0.94      0.94      0.94      5787
         24b       0.99      0.96      0.97       449
          25       0.90      0.67      0.77        70
          27       0.89      0.85      0.87       747
          28       0.93      0.76 

In [85]:
cm =confusion_matrix(y_test, y_pred)
pd.set_option('display.max_colwidth', None)
labels = np.unique(y_test)
df = pd.DataFrame(cm, index=labels, columns=labels)
print(df.to_string())

       02    03    04    05  06   07   10   11   16   20   21    23    24  24b  25   27   28   32    33
02   3501   183    24    20   0   23    2   49    1    8   29    18    50    0   0    4    5   24    70
03    155  1322    14    11   0    0    0    1    0    0    0     1     6    0   0    0    0    7     2
04     20    45  2287     2   0    2    0    5    0    0    2     0    13    0   0    0    0   33     6
05     32     8     4  2432   2   22    4   43    1    7   16    10    52    0   0   16    0    9    27
06      0     0     0     3  11    0    0    1    0    1    0     0     3    1   0    0    0    0     1
07     26     2     2    14   0  651   11   16    0    1    8    22    14    0   1    2    2    5     9
10      5     1     0    11   0    5  223    9    0    1    0     3    28    0   0    3    0    1     4
11      7     1     0     5   0    3    0  707    0    6    2    16    23    0   0    2   14    1     5
16     12     0     0     3   0    0    0    2  464    1    0   

**Word2Vec and Logistic Regression**

Use word similarities - word embeddingsconfusion_matrix

In [86]:
from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin.gz", binary=True)
wv.init_sims(replace=True)

TypeError: init_sims() got an unexpected keyword argument 'replce'

Let us find a weighted average of the vectors

In [93]:
import logging
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, token) for token in text_list ])

In [91]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\svenkata\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

* The following function applies word2vec average to the 'tokens' column
* This is taking very long time and throwing memory error
* Need to work on Mark Bell's powerful GPU

In [95]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
X1 = X[['tokens','ret_schedule']].copy()
train, test = train_test_split(X, test_size=0.3, random_state = 42)

test_tokenized = test.tokens #test.apply(lambda r: w2v_tokenize_text(r['tokens']), axis=1).values
train_tokenized = train.tokens #train.apply(lambda r: w2v_tokenize_text(r['tokens']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

MemoryError: Unable to allocate 18.0 GiB for an array with shape (16066345, 300) and data type float32

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(n_jobs=1, C=1e5)
log_reg = log_reg.fit(X_train_word_average, train['ret_schedule'])
y_pred = log_reg.predict(X_test_word_average)
print('accuracy %s' % accuracy_score(y_pred, test.ret_schedule))
print(classification_report(test.ret_schedule, y_pred,target_names=np.unique(test.ret_schedule)))

In [ ]:
cm =confusion_matrix(test.ret_schedule, y_pred)
pd.set_option('display.max_colwidth', None)
labels = np.unique(test.ret_schedule)
df = pd.DataFrame(cm, index=labels, columns=labels)
print(df.to_string())

**Doc2vec and Logistic Regression**

In [97]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled
X_train, X_test, y_train, y_test = train_test_split(X.tokens, X.ret_schedule, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [ ]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|████████████████████████████████████████████████████████████████████████| 93824/93824 [00:00<00:00, 106180.61it/s]


In [ ]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

**Finally some deep learning with keras**

In [ ]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
#import numpy as np
#import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
#from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_size = int(len(df) * .7)
train_posts = X['tokens'][:train_size]
train_tags = X['ret_schedule'][:train_size]

test_posts = X['tokens'][train_size:]
test_tags = X['ret_schedule'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 2

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)